In [1]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://haol-spark:7077") \
        .appName("Question_A1")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [8]:
eng = spark_context.textFile("/home/ubuntu/LDSA/ass_2/swe_eng/europarl-v7.sv-en.en")
swe = spark_context.textFile("/home/ubuntu/LDSA/ass_2/swe_eng/europarl-v7.sv-en.sv")

In [9]:
eng.count()

1862234

In [10]:
swe.count()

1862234

In [20]:
eng.take(2)

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.']

In [21]:
swe.take(2)

['Återupptagande av sessionen',
 'Jag förklarar Europaparlamentets session återupptagen efter avbrottet den 17 december. Jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester.']

In [16]:
engPiped = eng.pipe("/home/ubuntu/LDSA/ass_2/tools/tokenizer.perl -l en")
swePiped = swe.pipe("/home/ubuntu/LDSA/ass_2/tools/tokenizer.perl -l sv")

In [22]:
engPiped.take(2)

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999 , and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period .']

In [23]:
swePiped.take(2)

['Återupptagande av sessionen',
 'Jag förklarar Europaparlamentets session återupptagen efter avbrottet den 17 december . Jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester .']

In [18]:
engPiped.count()

1862234

In [24]:
swePiped.count()

1862234

In [33]:
engLowCase = engPiped.map(lambda line: line.lower().split("/n/n"))

In [34]:
sweLowCase = swePiped.map(lambda line: line.lower().split("/n/n"))

In [35]:
engLowCase.take(1)

[['resumption of the session']]

In [28]:
sweLowCase.take(1)

[['återupptagande av sessionen'],
 ['jag förklarar europaparlamentets session återupptagen efter avbrottet den 17 december . jag vill på nytt önska er ett gott nytt år och jag hoppas att ni haft en trevlig semester .']]

In [29]:
engLowCase.count()

KeyboardInterrupt: 

In [ ]:
sweLowCase.count()

In [ ]:
# release the cores for another application!
spark_context.stop()